In [ ]:
%pip install docling

In [ ]:
from docling.document_converter import DocumentConverter

source_file = "files/Resolucao_4.pdf"
converter = DocumentConverter()
result = converter.convert(source_file)

doc = result.document
markdown_content = doc.export_to_markdown()

with open("file.md", "w", encoding="utf-8") as file:
    file.write(markdown_content)

In [ ]:
!pip install -qU langchain-openai
!pip install -qU langchain_community unstructured
!pip install -qU langchain-text-splitters
!pip install -qU langchain-core
!pip install -qU "langchain-chroma>=0.1.2"
%pip install "unstructured[md]"

In [46]:
from dotenv import load_dotenv

load_dotenv()

True

In [47]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [48]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [49]:
from langchain_chroma import Chroma

In [50]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

In [51]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [52]:
def load_documents(file_path):
    
    print(f"Carregando documento: {file_path}")
    
    loader = UnstructuredMarkdownLoader(
        file_path,
        mode="single",
        strategy="fast",
    )

    documents = loader.load()
    
    print(f"Documento carregado com {len(documents[0].page_content)} caracteres")
    
    return documents

docs = load_documents("./data/leis/Lei_organica.md")

Carregando documento: ./data/leis/Lei_organica.md
Documento carregado com 162578 caracteres


In [ ]:
def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1200,
            chunk_overlap=150, # quanto o texto vai se sobrepor entre os chunks para continuar fazendo sentido
            separators=[
                "\nArt. ",
                "\nArtigo ",
                "\n§",
                "\nI -", "\nII -", "\nIII -",
                "\n\n",
                "\n",
                ". ",
                " "
            ]
        )
    
    chunks = text_splitter.split_documents(documents)
    print(f"Documento dividido em {len(chunks)} chunks")
    
    print(f"\n Exemplo do primeiro chunk:")
    print(f"{chunks[0].page_content[:100]}")
    
    return chunks

text_chunks = split_documents(docs)

In [53]:
"""Criando a vector store com ChromaDB"""
def create_vectorstore(chunks, persist_directory="./chroma_db"):
    
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small" # modelo de embedding otimizado para custo e desempenho
    )

    # Vector store, guarda os chunks dos documentos em vetores numéricos para facilitar a busca
    vectorstore = Chroma(
        collection_name="lei_organica",
        embedding_function=embeddings,
        persist_directory=persist_directory
    )

    vectorstore.add_documents(chunks)
    
    print(f"Vector store criado com {len(chunks)} documentos")
    
    return vectorstore

vectorstore = create_vectorstore(text_chunks)

Vector store criado com 229 documentos


In [ ]:
def setup_qa_chain(vectorstore):
    """Configuração da chain"""

    # Prompt
    template = """
    Você é um assistente especializado nas Leis do Município de Caçador/SC.
    Use o contexto abaixo para responder a pergunta.

    REGRAS:
        - Se não souber a resposta com base no contexto, diga que não sabe.
        - Utilize citações das leis quando possível.
        - Utilize até 30 linhas para a resposta.
        - Seja claro e direto, respondendo de forma concisa.

    Contexto:
    {context}

    Pergunta:
    {question}

    Resposta detalhada:"""

    prompt = PromptTemplate.from_template(template)

    # LLM
    llm = ChatOpenAI(
        #model="gpt-4o", 4,52s
        model="gpt-4o-mini", # 7s
        #model="gpt-3.5-turbo", 2,8s
        temperature=0
    )

    # Retriever, responsavel por buscar os documentos relevantes do contexto
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 5} # seleciona os 5 documentos/chunks mais relevantes
    )

    # Chain substituindo o RetrievalQA repassado pelo professor
    qa_chain = (
        {
            "context": retriever,
            "question": RunnablePassthrough()
        }
        | prompt
        | llm
    )

    print("Chain configurada com sucesso!")
    return qa_chain


qa_chain = setup_qa_chain(vectorstore)

Chain configurada com sucesso!


In [61]:
def ask_question(chain, question):
    print(f"\n {question}\n")
    
    response = chain.invoke(question)
    answer = response.content if hasattr(response, 'content') else str(response)
    
    print(f"{answer}")
    
    return

# Exemplos

In [56]:
ask_question(qa_chain, "Qual é a Subseção VI Das Comissões?")


 Qual é a Subseção VI Das Comissões?

A Subseção VI Das Comissões, conforme o contexto fornecido, está completamente revogada. Não há informações ou disposições atuais sobre essa subseção, uma vez que todos os seus artigos e parágrafos foram revogados pela Emenda à Lei Orgânica nº 15/2015. 

Portanto, não é possível fornecer detalhes sobre a Subseção VI, pois não existem textos ou normas vigentes a respeito. A revogação implica que as regras ou estruturas anteriormente estabelecidas nessa subseção não estão mais em vigor.


In [57]:
ask_question(qa_chain, "Quais são os poderes do município?")


 Quais são os poderes do município?

Os poderes do município de Caçador/SC estão delineados na Lei Orgânica Municipal, que estabelece diversas atribuições e responsabilidades. Entre os principais poderes, destacam-se:

1. **Legislar sobre assuntos de interesse local**: O município tem a competência de criar leis que atendam às necessidades e peculiaridades da sua população (Art. I).

2. **Cooperação com a União e o Estado**: O município deve atuar em colaboração com as esferas federal e estadual para o exercício das competências comuns, conforme previsto na Constituição Federal (Art. II).

3. **Suplementação da legislação**: O município pode suplementar a legislação federal e estadual, quando necessário (Art. III).

4. **Gestão orçamentária**: É responsabilidade do município elaborar o Plano Plurianual, a Lei de Diretrizes Orçamentárias e o Orçamento Anual (Art. IV).

5. **Instituição e arrecadação de tributos**: O município tem o poder de instituir e arrecadar tributos, além de aplic

In [58]:
ask_question(qa_chain, "Qual o percentual mínimo que o município deve aplicar em educação?")


 Qual o percentual mínimo que o município deve aplicar em educação?

O Município de Caçador/SC deve aplicar anualmente, no mínimo, 25% (vinte e cinco por cento) da receita resultante de impostos, incluindo a proveniente de transferências, na manutenção e desenvolvimento do ensino. Essa determinação está prevista no Art. 173 da Lei Orgânica do Município, que estabelece que "O Município aplicará anualmente 25% (vinte e cinco por cento), no mínimo da receita resultante de impostos, compreendida a proveniente de transferências, na manutenção e desenvolvimento do ensino, conforme art. 212 da Constituição Federal." 

Essa aplicação de recursos é fundamental para garantir a qualidade da educação no município, assegurando que as políticas educacionais sejam efetivas e atendam às necessidades da população.


In [62]:
ask_question(qa_chain, "Quem foi o campeão da copa do mundo de 1994?")


 Quem foi o campeão da copa do mundo de 1994?

Não sei. O contexto fornecido não contém informações sobre o campeão da Copa do Mundo de 1994.


In [63]:
ask_question(qa_chain, "Quem é o prefeito de Caçador em 2024?")


 Quem é o prefeito de Caçador em 2024?

Não sei quem é o prefeito de Caçador em 2024, pois essa informação não está disponível no contexto fornecido. A Lei Orgânica do Município de Caçador, que é a fonte do contexto, não menciona nomes de prefeitos ou outras autoridades específicas. Para obter essa informação, seria necessário consultar fontes atualizadas, como o site oficial da Prefeitura de Caçador ou outras fontes de notícias confiáveis.
